# 1.2 Free Options
# 1.2 Free Options
# 1.2 免费方案与零成本起步

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/forhow134/ai-coding-guide/blob/main/demos/01-first-experience/free_options.ipynb)

**预计 API 费用：$0（全部使用免费方案）**

本 Notebook 演示 4 种免费/低成本 LLM 方案，以及如何用统一代码切换不同提供商。

In [ ]:
!pip install -q openai google-genai

## 方案 1：Google AI Studio（Gemini）

In [ ]:
import os
from getpass import getpass

os.environ["GOOGLE_API_KEY"] = getpass("请输入 Google AI Studio API Key: ")

In [ ]:
from google import genai

client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="用一句话解释什么是 API",
)
print(f"[Gemini] {response.text}")

## 方案 2：OpenRouter 免费Models
<!-- 方案 2：OpenRouter 免费模型 -->

In [ ]:
os.environ["OPENROUTER_API_KEY"] = getpass("请输入 OpenRouter API Key: ")

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"],
)
response = client.chat.completions.create(
    model="deepseek/deepseek-chat-v3-0324:free",
    messages=[{"role": "user", "content": "用一句话解释什么是 API"}],
)
print(f"[OpenRouter/DeepSeek] {response.choices[0].message.content}")

## 方案 3：DeepSeek 直连（极低Cost）
<!-- 方案 3：DeepSeek 直连（极低成本） -->

In [ ]:
os.environ["DEEPSEEK_API_KEY"] = getpass("请输入 DeepSeek API Key（可选，回车跳过）: ")

In [ ]:
if os.environ.get("DEEPSEEK_API_KEY"):
    client = OpenAI(
        base_url="https://api.deepseek.com",
        api_key=os.environ["DEEPSEEK_API_KEY"],
    )
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[{"role": "user", "content": "用一句话解释什么是 API"}],
    )
    print(f"[DeepSeek] {response.choices[0].message.content}")
else:
    print("跳过 DeepSeek 直连测试")

## 方案 4：Ollama 本地Run（需本地安装）
<!-- 方案 4：Ollama 本地运行（需本地安装） -->

Ollama 需要在本地安装，Colab 上无法运行。以下代码展示调用方式：

```python
client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
response = client.chat.completions.create(
    model="qwen2.5:7b",
    messages=[{"role": "user", "content": "用一句话解释什么是 API"}],
)
print(response.choices[0].message.content)
```

本地安装步骤：
```bash
brew install ollama        # macOS
ollama run qwen2.5:7b      # 下载并运行模型
```

## 统一多提供商代码

一套代码，通过环境变量切换提供商——这是实际项目中的最佳实践。

In [ ]:
PROVIDERS = {
    "openai":     {"base_url": "https://api.openai.com/v1",    "model": "gpt-4o-mini"},
    "deepseek":   {"base_url": "https://api.deepseek.com",     "model": "deepseek-chat"},
    "openrouter": {"base_url": "https://openrouter.ai/api/v1", "model": "deepseek/deepseek-chat-v3-0324:free"},
    "ollama":     {"base_url": "http://localhost:11434/v1",    "model": "qwen2.5:7b"},
}

def ask(question: str, provider: str = "openrouter") -> str:
    config = PROVIDERS[provider]
    api_key_map = {
        "openai": os.environ.get("OPENAI_API_KEY", ""),
        "deepseek": os.environ.get("DEEPSEEK_API_KEY", ""),
        "openrouter": os.environ.get("OPENROUTER_API_KEY", ""),
        "ollama": "ollama",
    }
    client = OpenAI(base_url=config["base_url"], api_key=api_key_map[provider])
    response = client.chat.completions.create(
        model=config["model"],
        messages=[{"role": "user", "content": question}],
    )
    return response.choices[0].message.content

print(ask("用一句话解释什么是 API", provider="openrouter"))

## Hands-on Exercises
<!-- 动手练习 -->

1. 用 Google AI Studio 和 OpenRouter 分别回答同一个问题，对比回复质量
2. 修改 `ask()` 函数，添加 `Anthropic Claude` 作为新的 provider
3. 尝试在本地安装 Ollama 并运行 `qwen2.5:7b`，感受本地模型的速度和质量